# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [56]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    # TODO: implement function
    
    with open(infile) as f:
        content = f.readlines()    
 
    list_of_edges = []
    graph = nx.Graph()
    
    #Get the total number of nodes
    for line in content:
        if line[:1] == 'p':
            separated_p = line.split()
            number_of_nodes = int(separated_p[2])
    
    #Add the appropriate number of nodes
    for n in range(1, number_of_nodes + 1):
        graph.add_node(n)
    
    #Assign node demands
    for line in content:        
        if line[:1] == 'n':
            separated_n = line.split()
            graph.node[int(separated_n[1])]['demand'] = int(separated_n[2])
            
    #Create edges with capacities and weights
    #Keep new edges as they are. For multi-edges, create dummy nodes and dummy edges
    #All dummy edge capacities are equivalent to originals. First cost is equivalent, second is zero.
    for line in content:
        if line[:1] == 'a':
            separated_a = line.split()
            s1 = separated_a[1]
            s2 = separated_a[2]
            if (int(s1), int(s2)) not in graph.edges():
                graph.add_edge(int(s1), int(s2))
                graph.edge[int(s1)][int(s2)]['capacity'] = int(separated_a[4])
                graph.edge[int(s1)][int(s2)]['weight'] = int(separated_a[5])
            if (int(s1), int(s2)) in graph.edges():
                number_of_nodes += 1
                graph.add_node(number_of_nodes)                
                graph.add_edge(int(s1), number_of_nodes)
                graph.add_edge(number_of_nodes, int(s2))
                graph.edge[int(s1)][number_of_nodes]['capacity'] = int(separated_a[4])
                graph.edge[int(s1)][number_of_nodes]['weight'] = int(separated_a[5])
                graph.edge[number_of_nodes][int(s2)]['capacity'] = int(separated_a[4])
                graph.edge[number_of_nodes][int(s2)]['weight'] = 0
                
    graph = nx.DiGraph(graph)
    
    return graph

The following will check that your code outputs the expected min cost flow values on several test instances.

In [57]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [58]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    # TODO: implement function
    
    #Assign edges as variables
    vars = pulp.LpVariable.dicts("Route", G.edges(), None, None, pulp.LpInteger)
    
    node_supply = {}
    node_demand = {}
    
    #Create dictionaries for node supply and demand
    for n in G.nodes():
        if 'demand' in G.node[n]:
            if G.node[n]['demand'] > 0:
                node_supply[n] = 0
                node_demand[n] = G.node[n]['demand']
            elif G.node[n]['demand'] < 0:
                node_supply[n] = -G.node[n]['demand']
                node_demand[n] = 0
            else:
                node_supply[n] = 0
                node_demand[n] = 0
        else:
            node_supply[n] = 0
            node_demand[n] = 0
    
    #Create constraints from edge capacities
    for s1, s2 in G.edges():
        vars[(s1, s2)].bounds(0, G.edge[s1][s2]['capacity'])
        
    prob = pulp.LpProblem("Min Cost Flow HW4", pulp.LpMinimize)
    
    #Total Cost = Flow times Cost for each edge
    prob += pulp.lpSum([vars[(s1, s2)] * G.edge[s1][s2]['weight'] for s1, s2 in G.edges()]), "Total Cost"
    
    #Flow conservation constraint
    for node in G.nodes():
        prob += (node_supply[node] + pulp.lpSum([vars[(i,j)] for (i,j) in G.edges() if j == node]) >=
                 node_demand[node] + pulp.lpSum([vars[(i,j)] for (i,j) in G.edges() if i == node])), \
                "Flow Conservation in Node %s"%node
    
    prob.solve()
    
    return pulp.value(prob.objective)


The following will check that the LP finds the same optimal values as previously.

In [59]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
